# BIBA Exam Synopsis - AirBnb Listing Price Precition - 26/11/2018
By Begoña Sustatxa, Frederik Thulstrup, Tomas Nemecek, Yohannes Kidane kifle
## 1. Introduction
FairBnB is our imaginary AirBnb property management company. It helps customers renting out their properties at the correct price, managing customers, and offers cleaning services.
FairBnb has to precisely judge what type of properties to take on and what price point to rent out the propeties at, and consider what is their break-even point for each property considering FairBnb's expenses. 

To help make all these business decisions FairBnb has decided to explore AirBnb's listing data for properties in the city of London. 
FairBnb would like to automate the price prediction, by creating a model which would take all necessary factors into a consideration.   

To create the model, FairBnb first has to prepare and analyze the data, to see possible correlations between different factors. 

### 1.1 The dataset
We found the dataset on AirBnB's website: http://insideairbnb.com/get-the-data.html and chose the detailed listings dataset for London. It contains all information about each listing that is available for rent.
First let's start by importing it with the code below and running the str function to display the structure of the data.

In [1]:
listings_original <- read.csv("listings.csv")
str(listings_original)

'data.frame':	75506 obs. of  96 variables:
 $ id                              : int  9554 11076 13913 17402 24328 24903 25123 26223 26482 28010 ...
 $ listing_url                     : Factor w/ 75506 levels "https://www.airbnb.com/rooms/1000127",..: 74306 1670 8357 17376 43354 45451 46426 50311 51397 58667 ...
 $ scrape_id                       : num  2.02e+13 2.02e+13 2.02e+13 2.02e+13 2.02e+13 ...
 $ last_scraped                    : Factor w/ 1 level "2018-09-10": 1 1 1 1 1 1 1 1 1 1 ...
 $ name                            : Factor w/ 73287 levels "","  Shoreditch/BrickLane Sleeps 2/6",..: 24749 68860 35776 67746 8526 19304 19602 32704 32706 22081 ...
 $ summary                         : Factor w/ 68049 levels "","          Curious to experience what living in central London means? This is your ideal choice! Comfortable and"| __truncated__,..: 44748 54022 35873 41815 11404 21691 14158 18211 18211 25016 ...
 $ space                           : Factor w/ 47744 levels "","    A cosy 2 

As we can see there are a lot of properties for each listing and a lot of this information is usefull for us. But to use it properly we need to clean it.

## 2. Data cleaning
From the feature list above we can see that have 96 features and that there are many features that we do not need. There are also some features that have the wrong data type.   
We have made some assumptions regarding what columns we might require and have selected only those. We think these properties are the most crucial for the pricing of a listing. Comments in the code below describes the different ways we cleaned the data.

In [ ]:
install.packages("tidyverse")
library(tidyverse)

In [ ]:
# Get only the wanted features
wanted_features <- c("id","host_id","host_response_time","host_response_rate","neighbourhood_cleansed",
                     "property_type","room_type","accommodates","bathrooms","bedrooms","beds","bed_type",
                     "square_feet","price","security_deposit","cleaning_fee","extra_people","minimum_nights","maximum_nights",
                     "number_of_reviews","review_scores_rating","review_scores_accuracy","review_scores_cleanliness",
                     "review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value",
                     "instant_bookable","cancellation_policy","host_since")
listings <- listings_original[wanted_features]

In [ ]:
# Method to convert Features that are factor string with $ signs to numeric. 
# This is to clean the price property which is not in a numeric format and with a $ in the price.
priceFactorToNumeric <- function(column) {
    column = gsub("\\$", "", column)
    column = gsub("\\,", "", column)
    column = as.numeric(column)
    return (column)
}

# Covert price and fees from factor to numberic and set NA values to 0
listings$price <- priceFactorToNumeric(listings$price)

listings$cleaning_fee <- priceFactorToNumeric(listings$cleaning_fee)
listings$cleaning_fee[is.na(listings$cleaning_fee)] <- 0

listings$security_deposit <- priceFactorToNumeric(listings$security_deposit)
listings$security_deposit[is.na(listings$security_deposit)] <- 0

listings$extra_people <- priceFactorToNumeric(listings$extra_people)
listings$extra_people[is.na(listings$extra_people)] <- 0

# Convert from factors to numeric
listings$host_response_rate <- as.numeric(gsub("%", "", as.character(listings$host_response_rate)))/100

# rename neighbourhood_cleansed to neighborhood
colnames(listings)[which(names(listings) == "neighbourhood_cleansed")] <- "neighborhood"

In [ ]:
# Set NA values to 0. This might have a bad impact on our models since the values that are missing are unlikely to be 0. 
# We had to do it since there was very few listings without any NA values.
na_cols <- colnames(listings)[colSums(is.na(listings)) > 0]
for(name in na_cols){
    listings[name][is.na(listings[name])] <- 0
}

In [ ]:
# This is now the cleaned data.
str(listings)

After our cleaning process our dataset is significantly simplified and we can begin our data exploration and analysis. 

## 3. Exploratory data analysis
In this part we will explore some of the different properties to find possible correlations.

### 3.1 Room types
Lets first take a look at what type of rooms are being rented out the most. 

In [7]:
options(repr.plot.width=9, repr.plot.height=6)

# Get the room types and their percentages
room_types_counts <- table(listings$room_type)
room_types <- names(room_types_counts)
counts <- as.vector(room_types_counts)
percentages <- scales::percent(round(counts/sum(counts), 2))
room_types_percentages <- sprintf("%s (%s)", room_types, percentages)
room_types_counts_df <- data.frame(group = room_types, value = counts)

# Plot
pie <- ggplot(room_types_counts_df, aes(x = "", y = value, fill = room_types_percentages))+
  geom_bar(width = 1, stat = "identity")+
  coord_polar("y", start = 0)+
  scale_fill_brewer(palette = "Set1")+
  ggtitle("Room types")+
  ylab("")+
  xlab("")+
  labs(fill="")+
  theme(axis.ticks = element_blank(), panel.grid = element_blank(), axis.text = element_blank())+
  geom_text(aes(label = percentages), size = 7, position = position_stack(vjust = 0.5))
pie

From the pie chart we can see that most listings are Entire homes/apartments, with Private rooms being a close second place, and shared rooms being just 1%.

#### 3.1.1 Room types compared to price
Let's see  how the room type affects the price. To calculate this relationship, we group by the property type and compared it to the mean price.

In [8]:
listings %>%
  group_by(room_type) %>%
  summarise(mean_price = mean(price)) %>% 
  arrange(mean_price) %>% 
  ggplot() + 
  geom_bar(mapping = aes(x = reorder(room_type, +mean_price), y = mean_price, fill=room_type), stat = "identity") + 
  scale_fill_brewer("Room Type", palette = "Set1")+
  labs(title = "Room type to average price per night", x = "Room Type", y = "Average price per night in $") 

While the graph above is not exactly surprising, entire homes/apartments are the most expensive followed by private rooms. Guests obviously value their privacy and are willing to pay for it. 

### 3.2 Number of bedrooms
Next lets take a look at what number of listings offer what number of bedrooms

In [9]:
bedrooms_tbl <- table(listings$bedrooms)
bedrooms_df <- data.frame(nr_of_beds = names(bedrooms_tbl), 
                          value = as.vector(bedrooms_tbl))
bedrooms_df$nr_of_beds <- factor(bedrooms_df$nr_of_beds, levels=unique(bedrooms_df$nr_of_beds))
bedrooms_df

ggplot(data = bedrooms_df, aes(nr_of_beds, value))+
    geom_bar(stat = "identity", fill = "steelblue", width = 0.7) +
    labs(title = "Number of bedrooms offered", x = "# of bedrooms", y = "Listings")
    

nr_of_beds,value
0,4526
1,48033
2,15277
3,4992
4,1902
5,592
6,135
7,33
8,7
9,3


We can see see from the plot above that an overwhelming majority of listings offer just one bedroom. 

#### 3.2.1 Number of Bedrooms compared to price
Lets see how much of an impact the number of bedrooms have on the price. 

In [10]:
bedroomsdata <- filter(listings, !is.na(listings$bedrooms))
bedroomsdata %>%
  filter(room_type=="Entire home/apt")%>%
  group_by(bedrooms) %>%
  summarise(mean_price = mean(price)) %>% 
  arrange(bedrooms) %>% 
  ggplot() + geom_bar(mapping = aes(x = bedrooms, y = mean_price),fill = "steelblue", stat = "identity") +
  scale_x_continuous(breaks = 0:max(bedroomsdata$bedrooms)) + 
  labs(title = "Average price/night in $ to number of bedrooms", x = "Average price/night in $", y = "Average price/night in $")


We can see how much the price increases as the number of beds increases. From 0 (sofa) to 7 beds the price is increasing but afterwards the price becomes very varying. We think other factors comes in play afterwards. 

### 3.3 Neighborhood
Let us see how many listings are in which neighborhood, and how the neighborhood influences the price.

In [11]:
neigh_tbl <- table(listings$neighborhood)
neigh_df <- data.frame(neigborhood = names(neigh_tbl), 
                          value = as.vector(neigh_tbl))
neigh_df <- arrange(neigh_df, desc(value))
neigh_df

ggplot(data = neigh_df)+
    geom_bar(mapping = aes(x = reorder(neigborhood, +value), y = value), stat = "identity", fill = "steelblue", width = 0.7) + 
    labs(title = "Number of listings in a neighborhood", x = "Neighborhoods", y = "# of listings") +
    coord_flip()
    

neigborhood,value
Westminster,8047
Tower Hamlets,7384
Hackney,5775
Kensington and Chelsea,5344
Camden,5203
Islington,4714
Southwark,4518
Lambeth,4492
Hammersmith and Fulham,3861
Wandsworth,3834


We can see that the majority of listings are in Westminster and Tower Hamlets, with other neighborhoods being fairly far behind by a few thousand listings.     
Let's investigate how the neighborhood influences the price. 

#### 3.3.1 Neighborhood to price

In [12]:
neighbor_to_price <- listings %>%
  group_by(neighborhood) %>%
  summarise(mean_price = mean(price)) %>% 
  arrange(mean_price) %>% 
  ggplot() + geom_bar(mapping = aes(x = reorder(neighborhood, +mean_price), y = mean_price), fill = "steelblue", width = 0.7, stat = "identity") + 
  coord_flip() + labs(title = "Neighborhood to average price per night", x = "Neighborhood", y = "Average price per night in $")

neighbor_to_price

We can see that the three most expensive neighborhoods are **Kensington and Chelsea** and **Westminster** being at the top with nightly rent well over $200.  

**City of London** is a somewhat close 3rd with average $170 per night.    
One observation we can make is that Westminster is one of the most available neighborhoods with the largest amount of listings, however it is also second most expensive

#### 3.3.2 Neighborhood to square feet
Another correlation that might be worth investigating is between the property size the neighborhood. With it we can find out what is the average size of the property in a specific neighborhood, and thus whether our customer has an above average, average or below average property size than the competition.     
    
Since we replaces NA values in square_feet with 0, let's first remove those from the dataset. 

In [13]:
square_feet<-filter(listings, listings$square_feet > 0)
dim(square_feet)

[1] 444  30

As we can see that significantly reduced our dataset, to only 444 observations. Thefore that attribute probably will not have much effect on the price, however let's graph it anyway.

In [14]:
neighborhood_to_sq <- square_feet %>%
  group_by(neighborhood) %>%
  summarise(mean_square = mean(square_feet)) %>% 
  arrange(mean_square) %>% 
  ggplot() + geom_bar(mapping = aes(x = reorder(neighborhood, +mean_square), y = mean_square),fill = "steelblue", width = 0.7, stat = "identity") + 
  coord_flip() + labs(title = "Neighborhood to Square Feet", x = "Neighborhood", y = "Square Feet")
neighborhood_to_sq

Unfortunately, since the dataset does not actually contain many observations with valid square_feet value, we cannot really draw many conclusions from this correlations. 

### 3.4 Cleaning review to cleaning fee plot
With this relationship we wanted to find out if there was any connection betweeen the cleaning fee that the guests pay and the percieved cleanliness of the rented place. 

In [15]:
df_cleaning_fee <- filter(listings, !is.na(listings$cleaning_fee), !is.na(listings$review_scores_cleanliness))
df_cleaning_fee %>%
  group_by(review_scores_cleanliness) %>%
  summarise(mean_cleaning_fee = mean(cleaning_fee)) %>% 
  arrange(mean_cleaning_fee) %>% 
  ggplot() + geom_bar(mapping = aes(x = factor(review_scores_cleanliness), y = mean_cleaning_fee), fill = "steelblue", stat = "identity") + 
  labs(title = "Cleaning review to cleaning fee", x = "Cleaning review rating", y = "Average cleaning fee")


We found interesting results here as the review scores for cleaningness does not seem to depend on the cleaning fee. And the average price is around $25 regardless of the actual quality.

### 3.5 Data analysis conclusion
During the data analysis, we have mostly confirmed some of our assumptions about the dataset, such as that price is dependent on the neighborhood and number of bedrooms.

## 4. Modelling

### 4.1 Linear Regression
The main goal of our project is to be able to predict listing prices based on features that we input. We threfore need to find out which attributes have the biggest impact on price and then build a model based on them. We use the linear regression algorithm to create our models.     
First lets split our data into training and testing data.    

In [16]:
# We split our data to a training set and a test set
smp_size <- floor(0.9 * nrow(listings))
set.seed(357)
train_ind <- sample(seq_len(nrow(listings)), size = smp_size)
train <- listings[train_ind, ]
test <- listings[-train_ind, ]

Next we build a very simple model based on room_type and neighborhood and a slightly more complex model to see the model behavior.

In [17]:
simpleModel <- lm(data=train, price ~ room_type + neighborhood)
complexModel <- lm(data = train, price ~ room_type + neighborhood + 
    bedrooms + beds + number_of_reviews + bathrooms + accommodates + security_deposit)
summary(simpleModel)
summary(complexModel)


Call:
lm(formula = price ~ room_type + neighborhood, data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-205.2  -42.8  -13.0   10.1 9881.3 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                         118.2695     9.5980  12.322  < 2e-16 ***
room_typePrivate room               -82.7991     1.1381 -72.750  < 2e-16 ***
room_typeShared room               -100.6408     5.4729 -18.389  < 2e-16 ***
neighborhoodBarnet                   10.9499    10.3649   1.056 0.290771    
neighborhoodBexley                   -8.3482    14.1720  -0.589 0.555821    
neighborhoodBrent                     9.4511    10.0822   0.937 0.348556    
neighborhoodBromley                  -9.9104    11.4001  -0.869 0.384673    
neighborhoodCamden                   38.7436     9.7941   3.956 7.64e-05 ***
neighborhoodCity of London           55.7092    11.9148   4.676 2.94e-06 ***
neighborhoodCroydon                  -9.4963    10.7263  -0.885


Call:
lm(formula = price ~ room_type + neighborhood + bedrooms + beds + 
    number_of_reviews + bathrooms + accommodates + security_deposit, 
    data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-652.9  -32.6   -6.3   16.0 9888.1 

Coefficients:
                                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)                        -24.388941   9.031096  -2.701 0.006924 ** 
room_typePrivate room              -25.038518   1.302137 -19.229  < 2e-16 ***
room_typeShared room               -59.616794   5.241433 -11.374  < 2e-16 ***
neighborhoodBarnet                  10.204351   9.625310   1.060 0.289076    
neighborhoodBexley                  -7.338157  13.159481  -0.558 0.577097    
neighborhoodBrent                    6.848681   9.365436   0.731 0.464616    
neighborhoodBromley                  0.753422  10.586317   0.071 0.943263    
neighborhoodCamden                  44.203150   9.099220   4.858 1.19e-06 ***
neighborhoodCity of London          8

In [18]:
predSimple <- predict(simpleModel, test)
predComplex <- predict(complexModel, test)

predResSimple <- predSimple - test$price
predResComplex <- predComplex - test$price

test_errors <- data.frame(model = c("Simple", "Complex"), 
                         RMSE = c(sqrt(mean(predResSimple^2)), sqrt(mean(predResComplex^2))))
test_errors

model,RMSE
Simple,148.6358
Complex,140.3513


Above we can see that both models are not every good, as the Adjusted R-squared is around 13% and 25% for the simple and complex model, repectively. To see how all the variables impact the model, lets create a model based on all features. 

In [19]:
total_model <- lm(data = train, price ~ host_response_time + host_response_rate + neighborhood + property_type +
                 room_type + accommodates + bathrooms + bedrooms + beds + bed_type + security_deposit + cleaning_fee + 
                + extra_people + minimum_nights + maximum_nights + number_of_reviews + review_scores_rating + review_scores_accuracy +
               review_scores_cleanliness +  review_scores_checkin + review_scores_communication + review_scores_location + review_scores_value +
                instant_bookable + cancellation_policy)
summary(total_model)


Call:
lm(formula = price ~ host_response_time + host_response_rate + 
    neighborhood + property_type + room_type + accommodates + 
    bathrooms + bedrooms + beds + bed_type + security_deposit + 
    cleaning_fee + +extra_people + minimum_nights + maximum_nights + 
    number_of_reviews + review_scores_rating + review_scores_accuracy + 
    review_scores_cleanliness + review_scores_checkin + review_scores_communication + 
    review_scores_location + review_scores_value + instant_bookable + 
    cancellation_policy, data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-860.7  -31.9   -5.2   18.1 9644.7 

Coefficients:
                                                 Estimate Std. Error t value
(Intercept)                                     5.487e+01  4.594e+01   1.194
host_response_timea few days or more            2.883e+01  3.743e+01   0.770
host_response_timeN/A                           1.215e+01  3.729e+01   0.326
host_response_timewithin a day                  3.957e

As we can see the model with all the features as a slightly better Adjusted R-squared compared to our complex model, however the difference in Adjusted R-squared for such an increase in the number of features is negligible.     
From the summary we can see the most significant attributes based on the "star rating". Lets decrease the number of attributes by selecting only the most significant ones and create a new model. 

In [20]:
norm_model <- lm(data = train, 
            price ~ host_response_rate + neighborhood + property_type +
                 room_type + accommodates + bathrooms + bedrooms + beds + security_deposit + cleaning_fee + 
                review_scores_rating + review_scores_cleanliness + review_scores_value + cancellation_policy)
summary(norm_model)


Call:
lm(formula = price ~ host_response_rate + neighborhood + property_type + 
    room_type + accommodates + bathrooms + bedrooms + beds + 
    security_deposit + cleaning_fee + review_scores_rating + 
    review_scores_cleanliness + review_scores_value + cancellation_policy, 
    data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-862.6  -31.9   -5.2   17.9 9652.8 

Coefficients:
                                                 Estimate Std. Error t value
(Intercept)                                     5.544e+01  2.041e+01   2.717
host_response_rate                              2.336e+00  1.234e+00   1.893
neighborhoodBarnet                              9.474e+00  9.465e+00   1.001
neighborhoodBexley                             -5.049e+00  1.294e+01  -0.390
neighborhoodBrent                               6.911e+00  9.212e+00   0.750
neighborhoodBromley                            -3.516e+00  1.041e+01  -0.338
neighborhoodCamden                              4.182e+01  8.96

In [21]:
predTotal <- predict(total_model, test)
predNormal <- predict(norm_model, test)

predResTotal <- predTotal - test$price
predResNormal <- predNormal - test$price


test_errors <- data.frame(model = c("Simple", "Complex", "Total", "Normal"), 
                         RMSE = c(sqrt(mean(predResSimple^2)), sqrt(mean(predResComplex^2)), 
                                 sqrt(mean(predResTotal^2)), sqrt(mean(predResNormal^2))))
test_errors

model,RMSE
Simple,148.6358
Complex,140.3513
Total,136.2523
Normal,136.3276


## 5. Model Conclusion

Based on the model results we are not able to predict the price very accurately. Our best model is a model with all 30 features, and has around 28% accuracy. There are a number of possible issues with our model.
* The original data set has multiple shortcomings such as that the data is fairly incomplete with many values missing and some attributes are not included which could be significant. There are a lot of factors that are not in that dataset that might have a big influence on the price, but are hard to collect data about.
* We may have unintentionally corrupted the data during our data cleaning. Meaning our training data will be not very effective. 
* The features do not have a lot of predictive capability. It is possible that we have chosen the wrong features, however as we tested against a (most probably overfitted) model with all the features included and our models still fared badly in comparison. It is therefore possible that unfortunately a useful model is not very achievable based on this data set. 


## 6. Big data and storage issues discussion
In our case our dataset for AirBNB listings in London is not that big and therefore we do not really have the need for distributed databases. If our business was not only London based and we would have to mimic AirBnB’s database for all cities we would probably run into issues regarding the database size. Therefore it will be useful to use tools such as Hadoop or Spark. Not only is it a problem to store the data, we also need to perform some operations on it when data is added or updated. Therefore we need to use some tools to handle this. 

Right now we are thinking in terms of the listings data we got from AirBNB, but there is much more data than just this. Therefore if we got more data for example the reviews for each listing which we could do sentiment analysis on, we would be even more in need of Hadoop or Spark. These tools are smart because it moves the process to the data, instead of moving the data to the process. We need to calculate the average review score for a listing when a review is added. This is a small process in itself to calculate the average of some numbers, but if we had to get the data and calculate it, then it's a longer process. Therefore we would use Hadoop or Spark for our data. This is only in the case that we had to update data along the way and have the data for more cities. We would need these types of tools when we approach large datasets.

## 7. Ethical and legal issues
### 7.1 Legality and Ethicality of dataset usage
We do not foresee any legal issues regarding the usage of the listing data. This is data that AirBnb provided for public and commercial usage. If this data were for example secretly taken from their API, then there would be legal ramifications as it is AirBnB’s data. However this is not the case, as we mention above, all the data we are using is public on the AirBnB web. 
As to the ethicality of the dataset usage, we not see any potential points of contention either. Airbnb is fairly successful in obfuscating host personal data. We as a company do not have access to their exact addresses, banking informations etc. Therefore data that we are using cannot be easily misused.  
### 7.2 Legality and Ethicality of FairBnb’s business case
Regarding the legality of our business case, we do expect any legal problems. Our business case is using the data provided by AirBnb under the Creative commons license and analyze it to advise hosts how to make their listings more profitable, either by changing the price, or focusing on other changeable attributes, such as review scores etc. As this type of service is perfectly legal and also in accordance with AirBnB’s Terms and Conditions, as mentioned above, we do not see any legal issues.  
 
When talking about ethical issues, we believe that our service will actually bring apartment prices down, as it might increase the competition between hosts. Therefore by helping the host to make as much money as they can get from each reservation while still giving a fair price. This will have the consequence of lowering the prices for all guests. It is of course obvious that a apartment in the center of London is always going to be more expensive than one in the suburbs, but our goal is that the central apartment should not be overpriced just because of the location. As such we hope that our service would be a net positive for the AirBnB community.

## 8 Conclusion 

The purpose of this project was to predict the price of an AirBnb listings based on the parameters publicized by AirBnB. This prediction would allows to give advice to hosts on what they can change about to listings to maximize profit. 

Unfortunately, as mentioned above we were unable to create a reliable model based on the provided dataset. Our best model has the predictive accuracy (Adjusted R-squared) of 28%, which is very unreliable and one could not probably build a business on top of it. It will require better data and better models to give proper results.
